In [1]:
from pandas import DataFrame
import seaborn as sns
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import RobustScaler

from main import get_dataframe, iterate_mixedml, loop_mixedml

In [2]:
X_LABELS = ["x2_x5", "x4_x7", "x6_x8"]

# the paper says "y_fixed" for "y fixed effects"
# but it can be understood as "y not changing"
# so I call it "y_fe"
Y_LABEL = "y_mixed_obs"  # the values are fixed
Y_LABEL_FE = Y_LABEL + "_fixed"  # the value are not fixed

In [3]:
data = get_dataframe("../../data/synthetic_bph_1/01_test.csv")
data = data[data["individus"] < 20]
data = data[["individus"] + X_LABELS + [Y_LABEL]]
data[X_LABELS + [Y_LABEL]] = RobustScaler().fit_transform(data[X_LABELS + [Y_LABEL]])

In [4]:
data

,individus,x2_x5,x4_x7,x6_x8,y_mixed_obs
0,1,-0.802429,-0.471798,0.0,-1.129335
1,1,-0.744390,-0.323000,0.0,-0.932142
2,1,-0.716285,-0.249475,0.0,-0.896724
3,1,-0.700024,-0.222418,0.0,-0.858849
4,1,-0.690015,-0.214187,0.0,-0.905740
...,...,...,...,...,...
489,19,0.030368,-0.211567,0.0,-0.074280
490,19,0.030368,-0.211567,0.0,-0.071697
491,19,0.030368,-0.211567,0.0,-0.044650
492,19,0.030368,-0.211567,0.0,0.031557


In [5]:
??iterate_mixedml

Signature:
iterate_mixedml(
    ml_fixed: sklearn.neural_network._multilayer_perceptron.MLPRegressor,
    df_data: pandas.core.frame.DataFrame,
    lr: float,
) -> tuple[sklearn.neural_network._multilayer_perceptron.MLPRegressor, pandas.core.frame.DataFrame, bool]
Docstring: <no docstring>
Source:   
def iterate_mixedml(
    ml_fixed: MLPRegressor,
    df_data: DataFrame,
    lr: float,
) -> tuple[MLPRegressor, DataFrame, bool]:

    ####
    X = df_data[X_LABELS]
    y = df_data[Y_LABEL_FE]
    # we train ml_fixed by ignoring cluster effects (with the target y)
    ml_fixed.fit(X, y)
    # to get an estimate of y_fixed
    try:
        y_fixed = ml_fixed.predict(X)
    except AttributeError:
        y_fixed = ml_fixed.run(X)

    ####
    # based on e_fixed
    df_data["e_fixed"] = y - y_fixed
    df_data.to_csv("ml_pred.csv", index=False)
    # we estimate u
    call(["Rscript", "random_effects_fitting.R"])
    random_preds = read_csv("random_preds.csv")
    random_pred_ss = random_p

In [6]:
%cat random_effects_fitting.R

library(lcmm)

data <- read.csv2('ml_pred.csv', sep = ',', dec = '.')
data$x6_x8 <- as.numeric(data$x6_x8)


data$individus <- as.numeric(data$individus)
random_hlme <- hlme(
  e_fixed ~ 1,
  random = ~  x2_x5 + x4_x7 + x6_x8,
  idiag = TRUE,
  data = data,
  subject = 'individus',
)

write.table(
  random_hlme$pred,
  "random_preds.csv",
  sep = ",",
  dec = ".",
  row.names = FALSE
)

In [7]:
model = MLPRegressor(hidden_layer_sizes=(40, 30, 20), max_iter=5000)


results = {}
MAX_ITER = 100
for lr in [1e-1, 5e-1, 1]:
    results[lr] = loop_mixedml(model, data, eps=1e-3, lr=lr, max_iter=MAX_ITER)

mixedML step #01: 1.483951e-02
mixedML step #02: 1.220138e-02
mixedML step #03: 8.601417e-03
mixedML step #04: 1.143460e-02
mixedML step #05: 1.229519e-02
mixedML step #06: 1.012546e-02
mixedML step #07: 1.093765e-02
mixedML step #08: 1.112641e-02
mixedML step #09: 1.196993e-02
mixedML step #10: 1.067956e-02
mixedML step #11: 1.297431e-02
mixedML step #12: 1.038406e-02
mixedML step #13: 1.014369e-02
mixedML step #14: 1.098400e-02
mixedML step #15: 1.188298e-02
mixedML step #16: 1.306270e-02
mixedML step #17: 1.077843e-02
mixedML step #18: 1.054831e-02
mixedML step #19: 1.198870e-02
mixedML step #20: 1.389041e-02
mixedML step #21: 1.203549e-02
mixedML step #22: 1.081615e-02
mixedML step #23: 1.113364e-02
mixedML step #24: 1.006869e-02
mixedML step #25: 1.048457e-02
mixedML step #26: 1.446054e-02
mixedML step #27: 2.168414e-02
mixedML step #28: 1.216499e-02
mixedML step #29: 1.140656e-02
mixedML step #30: 1.128610e-02
mixedML step #31: 1.070111e-02
mixedML step #32: 1.237167e-02
mixedML 

In [10]:
{k: len(v) for k, v in results.items()}


{0.1: 100, 0.5: 100, 1: 100}

In [11]:
{k: max(v) for k, v in results.items()}

{0.1: np.float64(0.021684141133483967),
 0.5: np.float64(0.015896709463106107),
 1: np.float64(0.012822341121700222)}